In [19]:
import openai
import clip
import faiss
import torch
import numpy as np

In [24]:
def get_text_embedding(text):
    response = openai.embeddings.create(input=text, model="text-embedding-3-small")
    return response.data[0].embedding

def get_image_embedding(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, _ = clip.load("ViT-B/32", device=device)
    text_preprocessed = clip.tokenize([text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_preprocessed)
    return text_features.cpu().numpy().tolist()[0]

In [4]:
# Function to retrieve most relevant documents from a FAISS index
def retrieve_relevant_documents(query_embedding, index, top_k=5):
    query_embedding = np.array([query_embedding], dtype=np.float32)
    distances, indices = index.search(query_embedding, top_k)
    return indices[0]

In [6]:
text_index = faiss.read_index("textArchive.index")
image_index = faiss.read_index("imageArchive.index")

In [27]:
def queryOpenAI(query_text, image_index, text_index):
    query_text_embedding = get_text_embedding(query_text)
    query_clip_embedding = get_image_embedding(query_text)
    
    # Retrieve relevant texts and images
    text_indices = retrieve_relevant_documents(query_text_embedding, text_index)
    image_indices = retrieve_relevant_documents(query_clip_embedding, image_index)

    # Assuming you have text_documents and image_documents in lists or arrays
    relevant_texts = [text_documents[i] for i in text_indices]
    relevant_images = [image_documents[i] for i in image_indices]

    # Combine retrieved documents for context
    context = relevant_images + relevant_texts
    
    response = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages=[{
                "role": "user",
                "content": [
                    prompt
                ],
            }
            ]
        )
    
    return response

In [28]:
query = "Tell me about the ecosystem of the park"
queryOpenAI(query, image_index, text_index)

NameError: name 'text_documents' is not defined